In [1]:
import codecs
import lxml
from lxml import etree
import os
from mmap import mmap
import re
from os import listdir
from os.path import isfile, join
import MySQLdb
from datetime import datetime

In [2]:
class RootClass(object):
    """docstring"""
    
    def __init__(self, code, seatsAvail, authLevel, resHoldTotal):
        """Constructor"""
        self.code = code
        self.seatsAvail = seatsAvail
        self.authLevel = authLevel
        self.resHoldTotal = resHoldTotal
        
class Cabin(object):
    """docstring"""
    
    def __init__(self, rootClass, code, resHoldTotal, capacity, seatsAvail, allotment):
        """Constructor"""
        self.rootClass = rootClass
        self.code = code
        self.resHoldTotal = resHoldTotal
        self.capacity = capacity
        self.seatsAvail = seatsAvail
        self.allotment = allotment
        
class Leg(object):
    """docstring"""
    
    def __init__(self, cabins, dptDt, dptTime, arrDt, arrTime, orig, dest):
        """Constructor"""
        self.cabins = cabins
        self.dptDt = dptDt
        self.dptTime = dptTime
        self.arrDt = arrDt
        self.arrTime = arrTime
        self.orig = orig
        self.dest = dest
        
class SegCabin(object):
    """docstring"""
    
    def __init__(self, rootClass, code):
        """Constructor"""
        self.rootClass = rootClass
        self.code = code
        
class Segment(object):
    """docstring"""
    
    def __init__(self, segCabins, dptDt, orig, dest):
        """Constructor"""
        self.segCabins = segCabins
        self.dptDt = dptDt
        self.orig = orig
        self.dest = dest
        
class FlightId(object):
    """docstring"""
    
    def __init__(self, carrier, fltNr, opsSuffix, fltNrDptDt, fltNrArrDt):
        """Constructor"""
        self.carrier = carrier
        self.fltNr = fltNr
        self.opsSuffix = opsSuffix
        self.fltNrDptDt = fltNrDptDt
        self.fltNrArrDt = fltNrArrDt
        
class Flight(object):
    """docstring"""
    
    def __init__(self, flightId, legs, segments):
        """Constructor"""
        self.flightId = flightId
        self.legs = legs    
        self.segments = segments

In [3]:
def fast_iter(context, func, *args, **kwargs):
    for event, elem in iter(context):
        func(event, elem, *args, **kwargs)
        elem.clear()
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
    del context

In [4]:
def process_element(event, elem):
    if elem.tag == '{' + elem.nsmap.get(None) + '}FlightId' and elem.attrib.get('fltNr') + elem.attrib.get('opsSuffix') in fltNrs:
        flightId = FlightId(fltNrDptDt=elem.attrib.get('fltNrDptDt'), fltNrArrDt=elem.attrib.get('fltNrArrDt'), carrier=elem.attrib.get('carrier'), fltNr=elem.attrib.get('fltNr'),  opsSuffix=elem.attrib.get('opsSuffix'))
        
        legs = list()
        for leg in elem.getparent().findall('{' + elem.getparent().nsmap.get(None) + '}Leg'):
            cabins = list()
            for cabin in leg.findall('{' + leg.nsmap.get(None) + '}Cabin'):
                rootClassElem = cabin.find('{' + cabin.nsmap.get(None) + '}RootClass')
                if rootClassElem is not None:
                    rootClass = RootClass(code=rootClassElem.attrib.get('code'), seatsAvail=rootClassElem.attrib.get('seatsAvail'),authLevel=rootClassElem.attrib.get('authLevel'),resHoldTotal=rootClassElem.attrib.get('resHoldTotal'))
                    cabins.append(Cabin(rootClass=rootClass, code = cabin.attrib.get('code'),  resHoldTotal = cabin.attrib.get('resHoldTotal'), capacity = cabin.attrib.get('capacity'), seatsAvail = cabin.attrib.get('seatsAvail'), allotment = cabin.attrib.get('allotment')))
                else:
                    cabins.append(Cabin(rootClass=None, code = cabin.attrib.get('code'),  resHoldTotal = cabin.attrib.get('resHoldTotal'), capacity = cabin.attrib.get('capacity'), seatsAvail = cabin.attrib.get('seatsAvail'), allotment = cabin.attrib.get('allotment')))
            legs.append(Leg(cabins=cabins, arrDt=leg.attrib.get('arrDt'), dest=leg.attrib.get('dest'), dptTime=leg.attrib.get('dptTime'), arrTime=leg.attrib.get('arrTime'), dptDt=leg.attrib.get('dptDt'), orig=leg.attrib.get('orig')))

        segs = list()
        for seg in elem.getparent().findall('{' + elem.getparent().nsmap.get(None) + '}Segment'):
            cabins = list()
            for cabin in seg.findall('{' + seg.nsmap.get(None) + '}Cabin'):
                rootClassElem = cabin.find('{' + cabin.nsmap.get(None) + '}RootClass')
                if rootClassElem is not None:
                    rootClass = RootClass(code=rootClassElem.attrib.get('code'), seatsAvail=rootClassElem.attrib.get('seatsAvail'),authLevel=rootClassElem.attrib.get('authLevel'),resHoldTotal=rootClassElem.attrib.get('resHoldTotal'))
                    cabins.append(SegCabin(rootClass=rootClass, code = cabin.attrib.get('code')))
                else:
                    cabins.append(SegCabin(rootClass=None,  code = cabin.attrib.get('code')))
            segs.append(Segment(segCabins=cabins, dest=seg.attrib.get('dest'), dptDt=seg.attrib.get('dptDt'), orig=seg.attrib.get('orig')))

        proccess_flight(Flight(flightId=flightId, legs=legs, segments=segs) )

In [5]:
def proccess_flight(flight):
    
    # day timestamp
    reportDt = inventory_file.split("_")[1].split(".")[0]
    
    # insert flight
    cur.execute("INSERT INTO Flight(Carrier, fltNr, opsSuffix, fltNrDptDt, fltNrArrDt) VALUES(%s, %s, %s, %s, %s)", 
                (flight.flightId.carrier, int(flight.flightId.fltNr), flight.flightId.opsSuffix, 
                 datetime.strptime(flight.flightId.fltNrDptDt, "%d%m%y").date(), datetime.strptime(flight.flightId.fltNrArrDt, "%d%m%y").date()))
    flightId = cur.lastrowid
    
    # insert legs
    for leg in flight.legs:
        cur.execute("INSERT INTO Leg(DptDt, DptTime, ArrDt, ArrTime, orig, dest, fkFlightID) VALUES(%s, %s, %s, %s, %s, %s, %s)",
                (datetime.strptime(leg.dptDt, "%d%m%y").date(), datetime.strptime(leg.dptTime, "%H%M").time(), 
                 datetime.strptime(leg.arrDt, "%d%m%y").date(), datetime.strptime(leg.arrTime, "%H%M").time(),
                 leg.orig, leg.dest, int(flightId)))
        legId = cur.lastrowid
        
        # insert cabins
        for cabin in leg.cabins:
            cur.execute("INSERT INTO Cabin(Code, resHoldTotal, capacity, seatsAvail, allotment, fkLegID) VALUES(%s, %s, %s, %s, %s, %s)", 
                (cabin.code, float(cabin.resHoldTotal), int(cabin.capacity), int(cabin.seatsAvail), float(cabin.allotment), int(legId)))
            cabinId = cur.lastrowid
            
            # insert root if any
            if (cabin.rootClass != None):
                cur.execute("INSERT INTO RootClass(Code, seatsAvail, authLevel, resHoldTotal, fkCabinID, reportDt) VALUES(%s, %s, %s, %s, %s, %s)", 
                    (cabin.rootClass.code, int(cabin.rootClass.seatsAvail), 
                     float(cabin.rootClass.authLevel), float(cabin.rootClass.resHoldTotal), int(cabinId), datetime.strptime(reportDt, "%Y%m%d").date()))
            
    # insert segments
    for seg in flight.segments:
        cur.execute("INSERT INTO Segment(orig, dest, DptDt, fkFlightID) VALUES(%s, %s, %s, %s)", 
                (seg.orig, seg.dest, datetime.strptime(seg.dptDt, "%d%m%y").date(), int(flightId)))
        segId = cur.lastrowid

        # insert seg cabins
        for segCabin in seg.segCabins:
            cur.execute("INSERT INTO SegCabin(Code, fkSegmentID) VALUES(%s, %s)", 
                (segCabin.code, int(segId)))
            segCabinId = cur.lastrowid

            # insert root if any
            if (segCabin.rootClass != None):
                cur.execute("INSERT INTO RootClass(Code, seatsAvail, authLevel, resHoldTotal, fkSegCabinID, reportDt) VALUES(%s, %s, %s, %s, %s, %s)", 
                    (segCabin.rootClass.code, int(segCabin.rootClass.seatsAvail), 
                     float(segCabin.rootClass.authLevel), float(segCabin.rootClass.resHoldTotal), int(segCabinId), datetime.strptime(reportDt, "%Y%m%d").date()))


In [7]:
fltNrs = ('88J', '87J','102J', '101J', '337J', '337ZH', '50M', '49J', '10J', '9J', '138J', '137YQ')
db = MySQLdb.connect(host="194.247.15.117", 
                     port=7806,
                     user="bar",
                     passwd="PASSWORD",
                     db="Inventory")

inventory_path = '../inventory/rm-in/copy'
for inventory_file in [f for f in listdir(inventory_path) if isfile(join(inventory_path, f))]:
    if inventory_file.endswith('.xml'):
        db.autocommit(True) 
        cur = db.cursor()
        context = etree.iterparse(inventory_path + '/' + inventory_file)
        fast_iter(context, process_element)
        print(inventory_file + ' loading into db complete ' + str(datetime.now()))
        # db.close()
        cur.close()

inventory_20140901.xml loading into db complete 2017-07-14 02:08:27.974975
inventory_20140902.xml loading into db complete 2017-07-14 02:25:41.391172
inventory_20140903.xml loading into db complete 2017-07-14 02:43:35.821870
inventory_20140904.xml loading into db complete 2017-07-14 03:00:27.313254
inventory_20140905.xml loading into db complete 2017-07-14 03:16:53.945460
inventory_20140906.xml loading into db complete 2017-07-14 03:32:58.826059
inventory_20140907.xml loading into db complete 2017-07-14 03:48:33.320654
inventory_20140908.xml loading into db complete 2017-07-14 04:04:06.291978
inventory_20140909.xml loading into db complete 2017-07-14 04:19:26.286162
inventory_20140910.xml loading into db complete 2017-07-14 04:34:43.175237
inventory_20140911.xml loading into db complete 2017-07-14 04:49:54.992033
inventory_20140912.xml loading into db complete 2017-07-14 05:05:01.562221
inventory_20140913.xml loading into db complete 2017-07-14 05:19:59.366027
inventory_20140914.xml lo